In [1]:
import numpy as np

import sensors as sn
import em_tools as em

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 20
Num_emitters1 = 1
sample_size1 = 40
failing_sensors1 = np.arange(2,6)
gap_ratio1 = 0.5 * np.ones_like(failing_sensors1, dtype=np.float32)
theta1_rad = np.array([0.7]) # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q1 = 2.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sn.gss(Num_emitters1, sample_size1, P1)
N1 = sn.gss(Num_sensors1, sample_size1, Q1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 20
Num_emitters2 = 2
sample_size2 = 50
failing_sensors2 = np.array([2, 4], dtype=np.int16)
gap_ratio2 = 0.5 * np.ones_like(failing_sensors2, dtype=np.float32)
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q2 = 1.1 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sn.gss(Num_emitters2, sample_size2, P2)
N2 = sn.gss(Num_sensors2, sample_size2, Q2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sn.MCAR(X1, failing_sensors1, gap_ratio1)
X2_with_mv = sn.MCAR(X2, failing_sensors2, gap_ratio2)

In [5]:
X1_with_mv

array([[ 1.47375889e+00-1.19127215e+00j,  3.17520351e-02-1.88842948e+00j,
                    nan+0.00000000e+00j,  1.07070441e-01-1.01935785e+00j,
                    nan+0.00000000e+00j,             nan+0.00000000e+00j,
        -2.61303162e+00-4.42275651e-01j, -3.17165137e-01-2.00474714e+00j,
        -1.96754232e+00+7.76807750e-02j, -3.82252578e-01+7.73878140e-01j,
        -1.48424175e+00-2.02114333e+00j,  2.11749217e-01-3.72807955e-02j,
        -2.02449571e-01+1.24923104e+00j, -1.69654491e-01-1.14858969e+00j,
        -1.21912651e+00-8.19921326e-01j,  5.69511205e-01-4.33925179e-01j,
         8.68911479e-01-6.14488803e-01j, -7.63972860e-01-2.94873339e+00j,
         7.26624443e-01+2.73586965e-01j,  1.62576741e+00-6.19240956e-01j],
       [-7.50931649e-01-1.09293498e+00j, -9.99808654e-01-3.40390358e-01j,
         2.94445197e+00-1.56076135e+00j,  1.56124875e-01-9.21270290e-01j,
         7.60785961e-01+3.44869987e-01j,  2.89517460e-01-7.20109291e-01j,
         5.30210312e-02-4.30136133e-0

In [6]:
print(np.trace(sn.initial_Cov(X1_with_mv)))
print(np.trace(sn.complex_cov(X1)))

(54.03881916429678+0j)
(59.71827061996768+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em.multi_start_EM(X1_with_mv, 
                                                Num_emitters1, 
                                                Q=Q1, 
                                                num_of_starts=10, 
                                                max_iter=40, 
                                                rtol=1e-6,
                                                reg_coef=1e-3)

0-th start
theta=[0.15335215],P=[[0.00022361+0.j]]
likelihood is -1561.6627490142516 on iteration 0.
likelihood is -1561.6333899541826 on iteration 1.
likelihood is -1561.6043539573147 on iteration 2.
likelihood is -1561.5771115831396 on iteration 3.
likelihood is -1561.550535982112 on iteration 4.
likelihood is -1561.5247612641895 on iteration 5.
likelihood is -1561.5004745007961 on iteration 6.
likelihood is -1561.4780461952935 on iteration 7.
likelihood is -1561.4641726927853 on iteration 8.
1-th start
theta=[0.13636065],P=[[0.00022361+0.j]]
likelihood is -1561.6494026729367 on iteration 0.
likelihood is -1561.611839747118 on iteration 1.
likelihood is -1561.5766343896785 on iteration 2.
likelihood is -1561.5432251652614 on iteration 3.
likelihood is -1561.5112455800322 on iteration 4.
likelihood is -1561.4806457407387 on iteration 5.
likelihood is -1561.4519541267173 on iteration 6.
likelihood is -1561.426841102258 on iteration 7.
likelihood is -1561.4093455766465 on iteration 8.
L

In [8]:
print(f"EM_theta1_rad={EM_theta1_rad},\nEM_P1={EM_P1}")

EM_theta1_rad=[0.69602737],
EM_P1=[[1.79071+0.j]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.22761504],
Погрешность в смысле разности синусов углов:       [0.00304351].


In [10]:
print(np.trace(sn.initial_Cov(X2_with_mv)))
print(np.trace(sn.complex_cov(X2)))

(63.39917084790608+0j)
(66.81890096325793+0j)


In [11]:
EM_theta2_rad, EM_P2, lhd_2 = em.multi_start_EM(X2_with_mv, 
                                                      Num_emitters2, 
                                                      Q=Q2, 
                                                      num_of_starts=20, 
                                                      max_iter=20,
                                                      rtol=1e-3,
                                                      reg_coef=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[[0.00022361+0.00000000e+00j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.0324794 -7.21002848e-18j]]
1-th start
theta=[-1.43443568  0.13636065],P=[[0.00022361+0.0000000e+00j 0.        +0.0000000e+00j]
 [0.        +0.0000000e+00j 0.0264851 -1.0077221e-17j]]
2-th start
theta=[-0.16451812  1.40627821],P=[[0.00022361+0.00000000e+00j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.02436293-1.86231584e-17j]]
3-th start
theta=[-0.15355298  1.41724335],P=[[0.00384181-5.19242068e-17j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.02431452+2.34397769e-17j]]
4-th start
theta=[-1.04071883  0.5300775 ],P=[[0.00318056-8.00095738e-19j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.02954551+2.46207031e-18j]]
likelihood is -2951.776710497984 on iteration 0.
likelihood is -2609.683026891238 on iteration 1.
likelihood is -2381.5624698811444 on iteration 2.
likelihood is -2346.9226962749262 on iteration 3.
likeli

In [12]:
print(f"EM_theta2_rad={EM_theta2_rad},\nEM_P2={EM_P2}")

EM_theta2_rad=[-0.50037628  0.6998414 ],
EM_P2=[[2.28621687e+00+0.j        4.32095289e-04-0.3628276j]
 [4.32095289e-04+0.3628276j 1.68612436e+00+0.j       ]]


In [13]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.02155932 0.00908707],
Погрешность в смысле разности синусов углов:       [0.00033018 0.00012131].


In [14]:
em.incomplete_lkhd(X1_with_mv, theta1_rad, P1, Q1)

np.float64(-1342.180855971267)

In [15]:
em.incomplete_lkhd(X1_with_mv, EM_theta1_rad, EM_P1, Q1)

np.float64(-1349.0433769100505)

In [16]:
em.incomplete_lkhd(X2_with_mv, theta2_rad, P2, Q2)

np.float64(-1342.2713128492449)

In [17]:
em.incomplete_lkhd(X2_with_mv, EM_theta2_rad, EM_P2, Q2)

np.float64(-1351.3111008264677)